# Abberation mapping workflow 1: Process patches #

*25 Jan 2023*

*11 September 2023*

*27 September 2023*

Originally by Debayan Saha.  Jamie White modified Debayan's notebooks. 

Updating for Jamie's environment on the cluster.

Documenting installation / file locations.

Updating to run notebooks locally on a Mac / PC.  The psf modeling is super compute intensive, so it needs to be done on the cluster. Strategy: execute cluster commands via ssh.

## Environment paths ##

Not necessary in local non-cuda environment; transfer these to script executed via ssh.

In [1]:
# # not necessary for running locally on a mac  
# %env PATH=/sw/ap|ps/python3/3.7.4/bin:/sw/apps/jupyter-tensorboard/0.1.6/bin:/sw/apps/hdf5/1.8.16/bin:/sw/apps/cuda/10.0.130/bin:/sw/bin:/usr/lib64/qt-3.3/bin:/sbin:/bin:/usr/sbin:/usr/bin
# %env LD_LIBRARY_PATH=/sw/apps/python3/3.7.4/lib:/sw/apps/hdf5/1.8.16/lib:/sw/apps/cuda/10.0.130/lib64:/sw/apps/cuda/10.0.130/lib:/sw/lib

In [2]:
# ! env

## Load packages ##

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append(".")
# sys.path.append('/home/dsaha/.local/lib/python3.7/site-packages/')
from main import *
import os
import paramiko
import tifffile
import shutil
import warnings
import tqdm
from IPython.display import display, clear_output
from time import sleep

In [4]:
warnings.filterwarnings(action='once')
%load_ext autoreload
%autoreload 2

## Utility functions ##

In [5]:
def pretty_print_dict(d):
    print(json.dumps(d, indent=4))
    
def min_max_norm(arr):
    return (arr - np.min(arr))/(np.max(arr) - np.min(arr))

___

## Files and parameters ##

The notebook assumes everything is set locally and tries to reproduce the local project space on the remote `falcon` using the `paramiko` python package.

### Files ###

In [16]:
# relative path to bead image stack
image_file = "data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/beads.tif"

In [17]:
# fiji_path       = "/Applications/ImageJ/ImageJ.app/Contents/MacOS/ImageJ"      # absolute path to ImageJ/Fiji executable
# fiji_path = "/Applications/ImageJ.app/Contents/MacOS/ImageJ"
# fiji_command = "/usr/bin/java -jar /Applications/ImageJ.app/Contents/Java/ij.jar -batch"
remote_fiji_path = '/home/jwhite/fiji/Fiji.app/ImageJ-linux64' # location on cluster
remote_project_base_path = '/home/jwhite/projects/aberration_mapping/'
# relative path to fiji macros
fiji_macro_dir  =  "imageJ_macros/" 
# select proper macro
fiji_macro      = 'zola_macro_cluster_1p2_z_50_2ndfloor.ijm'# path to custom macros for running on cluster

In [18]:
zola_script_dir = 'scripts/'
zola_script     = 'bashscript_fiji.sh'

### Parameters ###

In [19]:
# set new parameters
crop_shape      = [40, 40, 40]
unit            = [0.05, 0.2, 0.2]
abb_list        = [3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
thresh_bck      = 10
num_folders     = 50
thresh_dist     = 11.29999999999999999999999999999999999
lam             = 0.515

## Construct local filepaths ##

From current working directory. 

In [20]:
local_base_path = os.getcwd()+'/' # everything relative to the notebook directory
local_base_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/'

In [21]:
local_fiji_macro_path = f'{local_base_path}{fiji_macro_dir}{fiji_macro}'
local_fiji_macro_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/imageJ_macros/zola_macro_cluster_1p2_z_50_2ndfloor.ijm'

In [22]:
# construct full path to image file and set save directory
local_img_path  = f"{local_base_path}{image_file}"
local_img_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/beads.tif'

In [23]:
# file_path and save_dir variables for compatibility
data_file_path = os.path.dirname(local_img_path) +'/'
save_dir  = data_file_path
data_file_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/'

In [36]:
local_zola_script_path = f"{local_base_path}{zola_script_dir}{zola_script}"
local_zola_script_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/scripts/bashscript_fiji.sh'

## Construct remote filepaths ##

In [29]:
remote_project_base_path

'/home/jwhite/projects/aberration_mapping/'

In [30]:
remote_image_file_path = f"{remote_project_base_path}{image_file}"
remote_image_file_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/beads.tif'

In [32]:
remote_fiji_macro_path = f'{remote_project_base_path}{fiji_macro_dir}{fiji_macro}'
remote_fiji_macro_path

'/home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_1p2_z_50_2ndfloor.ijm'

In [33]:
# construct full path to image file and set save directory
remote_img_path  = f"{remote_project_base_path}{image_file}"
remote_img_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/beads.tif'

In [34]:
# file_path and save_dir variables for compatibility
data_file_path = os.path.dirname(remote_img_path) +'/'
save_dir  = data_file_path
data_file_path

'/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/'

In [37]:
remote_zola_script_path = f"{remote_project_base_path}{zola_script_dir}{zola_script}"
remote_zola_script_path

'/home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh'

## Construct new parameters dictionary ##

In [38]:
# parameters
new_parameters = {}

new_parameters['crop_shape']      = crop_shape
new_parameters['unit']            = unit
new_parameters['abb_list']        = abb_list
new_parameters['FIJI_MACRO_PATH'] = fiji_macro_path
new_parameters['thresh_bck']      = thresh_bck
new_parameters['num_folders']     = num_folders
new_parameters['thresh_dist']     = thresh_dist
new_parameters['lam']             = lam
new_parameters

NameError: name 'fiji_macro_path' is not defined

## Make sure everything we need is on the remote

In [48]:
# local project space will be reconstructed relative to this directory on the remote
remote_project_base_path

'/home/jwhite/projects/aberration_mapping/'

In [50]:
# compare to local project space
local_base_path

'/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/'

In [ ]:
# check that the project folder is the same on both systems
local_project_check = 

ssh remote_host "ls > /tmp/file_on_remote_host.txt"

In [41]:
file_path_test = '/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2'
file_path_test = '/Users/jwhite/Library/Mobile Documents/com~apple~CloudDocs/Science/Grill_Lab/Projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/'
file_path_test = 'projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2'
! ssh falcon "ls -lga {file_path_test}"

total 324156
drwxr-sr-x  9 project-jwhite      4096 Sep 19 16:00 .
drwxrwsr-x 12 project-jwhite      4096 Sep 19 11:48 ..
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 abb_maps
-rw-r--r--  1 project-jwhite      2396 Sep 19 16:00 accepted_points_in_image.png
-rw-r--r--  1 project-jwhite     51343 Sep 19 15:59 amplitude_distribution.png
-rwxr--r--  1 project-jwhite 321626563 Sep 19 11:48 beads.tif
-rwxr--r--  1 project-jwhite      4096 Sep 19 11:50 ._.DS_Store
-rwxr--r--  1 project-jwhite      6148 Sep 20 10:49 .DS_Store
-rw-r--r--  1 project-jwhite  10072554 Sep 19 16:00 label_map.tif
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 measurements
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 med_measurement
-rw-r--r--  1 project-jwhite       317 Sep 19 11:48 params.json
drwxr-sr-x 53 project-jwhite      4096 Sep 19 13:37 patches_for_zola
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 point_boxes
-rw-r--r--  1 project-jwhite     15128 Sep 19 16:00 pointlist.npy
drwxr-s

In [42]:
! ssh falcon "ls -lga" {file_path_test}

total 324156
drwxr-sr-x  9 project-jwhite      4096 Sep 19 16:00 .
drwxrwsr-x 12 project-jwhite      4096 Sep 19 11:48 ..
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 abb_maps
-rw-r--r--  1 project-jwhite      2396 Sep 19 16:00 accepted_points_in_image.png
-rw-r--r--  1 project-jwhite     51343 Sep 19 15:59 amplitude_distribution.png
-rwxr--r--  1 project-jwhite 321626563 Sep 19 11:48 beads.tif
-rwxr--r--  1 project-jwhite      4096 Sep 19 11:50 ._.DS_Store
-rwxr--r--  1 project-jwhite      6148 Sep 20 10:49 .DS_Store
-rw-r--r--  1 project-jwhite  10072554 Sep 19 16:00 label_map.tif
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 measurements
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 med_measurement
-rw-r--r--  1 project-jwhite       317 Sep 19 11:48 params.json
drwxr-sr-x 53 project-jwhite      4096 Sep 19 13:37 patches_for_zola
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 point_boxes
-rw-r--r--  1 project-jwhite     15128 Sep 19 16:00 pointlist.npy
drwxr-s

In [43]:
! ssh falcon "cd projects"

In [44]:
! ssh falcon "cd projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/; pwd; ls -lga"

/home/jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2
total 324156
drwxr-sr-x  9 project-jwhite      4096 Sep 19 16:00 .
drwxrwsr-x 12 project-jwhite      4096 Sep 19 11:48 ..
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 abb_maps
-rw-r--r--  1 project-jwhite      2396 Sep 19 16:00 accepted_points_in_image.png
-rw-r--r--  1 project-jwhite     51343 Sep 19 15:59 amplitude_distribution.png
-rwxr--r--  1 project-jwhite 321626563 Sep 19 11:48 beads.tif
-rwxr--r--  1 project-jwhite      4096 Sep 19 11:50 ._.DS_Store
-rwxr--r--  1 project-jwhite      6148 Sep 20 10:49 .DS_Store
-rw-r--r--  1 project-jwhite  10072554 Sep 19 16:00 label_map.tif
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 measurements
drwxr-sr-x  2 project-jwhite      4096 Sep 19 16:00 med_measurement
-rw-r--r--  1 project-jwhite       317 Sep 19 11:48 params.json
drwxr-sr-x 53 project-jwhite      4096 Sep 19 13:37 patches_for_zola
drwxr-sr-x  2 project-jwhite      4096 Sep 

In [46]:
! echo -e "cd projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2\nget beads.tif" | sftp falcon

Connected to falcon.
sftp> cd projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2
sftp> get beads.tif
Fetching /lustre/projects/project-jwhite/projects/aberration_mapping/data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test2/beads.tif to beads.tif
beads.tif                                     100%  307MB  40.0MB/s   00:07    


## Fiji application and ImageJ macros

### Check Fiji macros ###

In [ ]:
# check current macro directory
fiji_macro_dir

In [ ]:
fiji_macro_path

In [ ]:
# check available macros
! ls -lga "{fiji_macro_dir}"

## Check old parameters if they exist

In [ ]:
# look at parameters from last processing job
old_params      = setup_param_file(img_path)
type(old_params)
old_params

## Load and show the image

In [ ]:
img = tifffile.imread(img_path)
img_max = np.max(img,0)

%matplotlib inline
fig, axs = plt.subplots(1,1,figsize=(10,7))
plt.imshow(img_max, clim=(np.percentile(img_max,0), np.percentile(img_max,99)))
plt.title("Full image");
# None;

## Load points from csv file and save patches

In [ ]:
from main import *
find_psf_crops(
    img_path, 
    crop_shape     = crop_shape, 
    num_folders    = num_folders, 
    thresh_bck     = thresh_bck, 
    threshold_dist = thresh_dist,
    unit           = unit,
    show_ignored   = False, 
    show_accepted  = False, 
    headless       = False
)

## Load patches

In [ ]:
# load the patches
patch_files = glob.glob(f"{os.path.dirname(img_path)}/patches_for_zola/patches*/planes_10_*.tif")
print(f"Found {len(patch_files)} patch files")
names = [os.path.splitext(os.path.basename(p))[0] for p in patch_files]
crops = {n: tifffile.imread(p) for n, p in zip(names, patch_files)}
pos = np.array([list(map(int, re.findall(r'\d+', n))) for n in names])
pos = pos[:,1:]

In [ ]:
patch_files[:2]

## Show the beads

In [ ]:
i=0
for n,pat in crops.items():
    zz,yy,xx = pat.shape
    x_arr = np.arange(-1*yy//2, yy//2)
    z_arr = np.arange(-1*zz//2, zz//2)
    fig, axs = plt.subplots(1,2,figsize=(7,3))
    print(n, np.max(pat) - np.median(pat))
    # print(shannon_entropy(pat))
    axs[0].imshow(np.max(pat,0))
    im = axs[1].imshow(np.max(pat,1))
    plt.colorbar(im)
    plt.show()
    print(pat.shape)
    i=i+1
    if i==50:
        break
None;

In [ ]:
plt.imshow(img_max, clim=(np.percentile(img_max,3), np.percentile(img_max,99)))
plt.plot(pos[:,2], pos[:,1],'r.');

In [ ]:
snr = []
for key,val in crops.items():
    val = min_max_norm(val)
    snr.append(np.max(val) - np.median(val))
snr = np.array(snr)
plt.hist(snr.flatten());

## Run Zola macro via bash command

Needs fiji for linux. Copy `Fiji.app` from Debayan's folder.

Uses `https://github.com/imodpasteur/ZOLA-3D` plugin: put the file

`ZOLA_-0.2.8-SNAPSHOT.jar`

into the plugins directory.


### Troubleshooting ###

**Test Deb's ImageJ macro**

In [ ]:
i = 29
INPUT_DIR=f"{save_dir}patches_for_zola/patches_{i}"
fiji_command = "/usr/bin/java -jar /Applications/ImageJ.app/Contents/Java/ij.jar -batch"
# fiji_command = "/usr/bin/java -jar /Applications/ImageJ.app/Contents/Java/ij.jar -run"
fiji_path = '/home/jwhite/fiji/Fiji.app/ImageJ-linux64 ' # location on cluster
print(f"{INPUT_DIR}")
print()
print(f"{fiji_macro_path}")
print()
# print(f'{fiji_path}')
print(f"{fiji_command}")
print()
print(f'{fiji_command}  "{fiji_macro_path}"  "{INPUT_DIR}"')

In [ ]:
# ! {fiji_command}  "{fiji_macro_path}"  "{INPUT_DIR}"

In [ ]:
! ssh falcon  "sbatch /home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh /home/jwhite/projects/aberration_mapping/imageJ_macros/zola_macro_cluster_1p2_z_50.ijm  '/home/jwhite/projects/aberration_mapping/data/andrea/0p5mkBeadsDICHmirror_4/patches_for_zola/patches_28/' "

In [ ]:
! ssh falcon "squeue -u jwhite"

In [ ]:
# local_macro_for_remote = 'imageJ_macros/zola_macro_cluster_1p2_z_50.ijm'
# fiji_path = '/home/jwhite/fiji/Fiji.app/ImageJ-linux64'

fiji_macro_path = 'imageJ_macros/zola_macro_cluster_1p2_z_50_2ndfloor.ijm'

zola_script_path = 'scripts/bashscript_fiji.sh'

INPUT_DIR = "data/andrea/2022_09_06_Beads_0p2_2ndfloorscope_test/patches_for_zola/patches_29"

print(f"sbatch  {zola_script_path} {fiji_macro_path} {INPUT_DIR}")

In [ ]:
print(f"cd projects/aberration_mapping; sbatch {zola_script_path} {fiji_macro_path} {INPUT_DIR}")

In [ ]:
! ssh falcon "cd projects/aberration_mapping; sbatch {} {zola_script_path} {fiji_macro_path} {INPUT_DIR}"

In [ ]:
! ssh falcon "squeue -u jwhite"

In [ ]:
! ls -lga

In [ ]:
! cat scripts/bashscript_fiji.sh | ssh falcon sbatch

# !sbatch /home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh {FIJI_MACRO_PATH} {INPUT_DIR}

In [ ]:
# be sure the std/ directory exists
# !sbatch  -J {i} -e std/{i}.err -o std/{i}.out /home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh {FIJI_MACRO_PATH} {INPUT_DIR}

### Run Zola analysis on cluster ###

In [ ]:
# Deleting the zola_raw folder and creating a new one
shutil.rmtree(f"{save_dir}/patches_for_zola/zola_raw/", ignore_errors=True)
os.makedirs(  f"{save_dir}/patches_for_zola/zola_raw/")

INPUT_DIR=f"{save_dir}/patches_for_zola/patches_{i}"

In [ ]:
# check what we are sending to command line
! echo {zola_script_path}
print()
! echo {FIJI_MACRO_PATH}
print()
! echo {INPUT_DIR} 

In [ ]:
# not sure why Debayan had jobs split into two groups
#
# ***** be sure the std/ directory exists *****
#
# running zola on all the patches
for i in range(35):
# for i in [28]:
# for i in range(1):
# for i in range(0,num_folders):
    FIJI_MACRO_PATH = fiji_macro_path
    INPUT_DIR=f"{save_dir}/patches_for_zola/patches_{i}"
    !sbatch -J {i} -e std/{i}.err -o std/{i}.out {zola_script_path} {FIJI_MACRO_PATH} {INPUT_DIR}
    # print()
#     break

for i in range(35, num_folders):
    INPUT_DIR=f"{save_dir}/patches_for_zola/patches_{i}"
    print(INPUT_DIR)
    print(zola_script_path)
    !sbatch -J {i} -e std/{i}.err -o std/{i}.out {zola_script_path} {FIJI_MACRO_PATH} {INPUT_DIR}
    print()

In [ ]:
l = !ssh falcon "squeue -u jwhite"
l = l[1:]
l

In [ ]:
# monitor progress....
l = !ssh falcon "squeue -u jwhite"
l = l[1:]
n = len(l)
while n > 0:
    clear_output(wait=False)
    l = !ssh falcon "squeue -u jwhite"
    l = l[1:]
    n=len(l)
    display(f'{n} files to go')
    sleep(1)

## Write current parameters to `params.json`

In [ ]:
# Write to params file
params = {
    'crop_shape':crop_shape, 
    'unit':unit, 
    'abb_list': abb_list, 
    'FIJI_MACRO_PATH': FIJI_MACRO_PATH, 
    'COMPUTERNAME' : socket.gethostname(),
    'thresh_bck':thresh_bck, 
    'lam':lam, 
    'thresh_dist':thresh_dist
}

write_to_param_file(img_path, params)

## Sample Fiji macro

```
/*
 * @Debayan MPI-CBG
 */

file_path = getArgument();
print("In macro");
print(file_path);
processFolder(file_path); 

// function to scan folders/subfolders/files to find files with correct suffix
function processFolder(input) {
	start = getTime();
	print(input)
	list = getFileList(input);
	list = Array.sort(list);
	print(list.length)
	for (i = 0; i < list.length; i++) {
		print(list[i]);
		if(File.isDirectory(input + File.separator + list[i])){
			l = substring(list[i], 0, lengthOf(list[i])-1);
			processFolder(input + File.separator + l + File.separator);
		}
		if(endsWith(list[i], ".tif")){
			processFile(input, list[i]);
		}
	}
	print("Totoal Time: "+(getTime()-start)/1000);   
}

function processFile(input, file) {
	print("Processing: "+ input + File.separator + file);
	if (indexOf(input, "patches") > 0) {
		if (!File.exists(File.getParent(input) + File.separator + "zola_raw")){
			File.makeDirectory(File.getParent(input) + File.separator + "zola_raw"); 
		}
	output = File.getParent(input) + File.separator + "zola_raw"+ File.separator + substring(file, 0, indexOf(file, '.tif'))+'.json';
	print(output);

	open(input + File.separator + file);
	setSlice((nSlices+1)/2); 
	makePoint(getWidth()/2, getHeight()/2);
	wid = 64;
	run(" Calibration: PSF modeling", "run_on_gpu gain=1 pixel_size=200 z_step=50 bead_moving=[far -> close to objective] numerical_aperture=1.2 immersion_refractive=1.33 wavelength=0.515 patch_size="+wid+" zernike_coefficient=[Zernike 15 coefs] iteration=20 result_calibration_file="+output);
	run("Close All");
	}
	else{print("No patches");
	}
}

```

## Sample Zola bash scripts ##



**`/projects/project-dsaha/insitu_psf/bashscript_fiji.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 8:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --mem 1000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/projects/project-dsaha/Fiji_jobs/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"

```

**`/home/jwhite/projects/aberration_mapping/scripts/bashscript_fiji.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 8:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=gpu
#SBATCH --gres=gpu:1
#SBATCH --mem 1000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/home/jwhite/fiji/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"


```

**`/projects/project-dsaha/insitu_psf/bashscript_fiji_short.sh`**

```
#!/bin/bash
#SBATCH -J "One"
#SBATCH -t 1:00:00
#SBATCH -n 1
#SBATCH -N 1
#SBATCH --partition=short
#SBATCH --gres=gpu:1
#SBATCH --mem 5000


#export OPENBLAS_NUM_THREADS=4

module unload cuda
module load cuda/10.0.130
#module load cuda/9.0.176

echo "Starting Fiji script"


FIJI_MACRO_FILEPATH=$1
PATCHES_FOLDER_PATH=$2
echo ${FIJI_MACRO_FILEPATH}
echo ${PATCHES_FOLDER_PATH}
echo "Bashscript"
/projects/project-dsaha/Fiji_jobs/Fiji.app/ImageJ-linux64 --headless -batch ${FIJI_MACRO_FILEPATH} ${PATCHES_FOLDER_PATH}

echo "Finishing Fiji script"


```

# End #